# ETL 
## Requirements

In [ ]:
pip install pandas openai requests

In [61]:
# Replace with you openai key
%set_env OPENAI_KEY = sk-1234567890

SyntaxError: invalid syntax (3833762931.py, line 1)

## Extracting

In [15]:
import pandas as pd
import requests

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()

sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

def get_user(id):
    response = requests.get(f'{sdw2023_api_url}/users/{id}')
    return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]

## Transforming

In [58]:
import openai
import os

openai.api_key = os.environ['OPENAI_KEY']

def generate_ai_news(user):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a fintech marketing specialist"},
            {"role": "user", "content": f"create a personal message to the fintech client {user['name']}. Tell to him about how important investinnments are. The message size should be 100 characters"}
        ]
    )

    return completion.choices[0].message.content.strip('\"')

for user in users:
    news = generate_ai_news(user)
    user['news'].append({
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pix.svg",
        "description": news
    })

## Loading

In [59]:
def update_user(user):
    response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
    return True if response.status_code == 200 else False

for user in users:
    success = update_user(user)
    print(f"User {user['name']} updated? {success}!")

User Pyterson updated? False!
User Pip updated? True!
User Pep updated? True!
